# First Layer

In [168]:
from selenium import webdriver
from bs4 import BeautifulSoup
import urllib.request
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains

import datetime
import time
import pandas as pd
import numpy as np
from datetime import date
date_today = date.today()

In [169]:
product_name = []
original_price = []
sale_price = []
product_url = []
picture_url = []

# in order to only crawler men's cloth info
link1 = 'https://www.massimodutti.com/us/men/sale/selected-items-up-to--50%25-c1887163.html'
# range(0,5); Sales: selected - trousers & denim
link2 = 'https://www.massimodutti.com/us/men/sale/polo-shirts-%26-t-shirts-c1748196.html'
# range(0,5); Sales: polo - formal shirts
link3 = 'https://www.massimodutti.com/us/men/new-in-c1866501.html'
# range(1), New in
link4 = 'https://www.massimodutti.com/us/men/collection/coats-%26-jackets-c1884501.html'
# range(0,6); Collections: Coats - Polos
linklist = [link1, link2, link3, link4]
rangelist = [range(5), range(5), range(1), range(6)]

In [170]:
for i in range(4):
    driver = webdriver.Chrome('/Users/fufufu/Documents/Python/Web Analytics/Selenium/chromedriver')
    driver.get(linklist[i])
    time.sleep(4) # cookie notification needs time to load
    driver.find_element_by_xpath("/html/body/div[2]/div[3]/div/div/div/div/div/div[2]/button").click() # close coookie notification
    driver.find_element_by_xpath('/html/body/itx-catalog/section/div/category-page/section[1]/div[1]/category-toolbar/div/ul/li[2]/a[2]/i').click()
    # scroll page
    for j in rangelist[i]:
        last_height = driver.execute_script("return document.body.scrollHeight") # Get scroll height
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            driver.execute_script("window.scrollTo(0, -1);") # need to scroll a bit up or the page wont load
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            # Wait to load page
            time.sleep(3)
            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                # till new_height = last_height, end loop, meaning cant scroll web page anymore, ie we are at end of this page
                break 
            last_height = new_height
        # at end of this page, get info from this page
        soup = BeautifulSoup(driver.page_source,'html.parser')
        for i in soup.find_all('itx-product-view', {'ng-if':'::(product.state.isProduct || product.state.isFakeBanner)'}):
            if i.find('a', {'ng-bind':'::$ctrl.product.name'}).text != '':
                product_name.append(i.find('a', {'ng-bind':'::$ctrl.product.name'}).text)  # product name
                if i.find('span', {'class':'old-price text-14-m ng-binding ng-scope'}) == None:
                    # if it == None, No sale price, 
                    # if it != None, sale price is sale price, original price is original price is original price
                    original_price.append(i.find('span', {'class':'price text-14-m ng-binding ng-scope'}).text) # original
                    sale_price.append(None)
                else:
                    original_price.append(i.find('span', {'class':'old-price text-14-m ng-binding ng-scope'}).text) # original
                    sale_price.append(i.find('span', {'class':'price text-14-m ng-binding ng-scope'}).text) # sale
                product_url.append(i.find('a', {'ng-click':'$ctrl.goToProductDetail($event)'}).get('ng-href')) # product url
                picture_url.append(i.find('img', {'class':'card9-image ng-scope'}).get('ng-src')) # pic url
            else:
                product_name.append(None) # product name
                original_price.append(None) # original price
                sale_price.append(None) # sale price
                product_url.append(None) # product url
                picture_url.append(None) # picture url
        try: # click the button to next page
            driver.find_element_by_xpath('/html/body/itx-catalog/section/div/category-page/section[1]/div[2]/category-nav/div/div[2]/a').click()
        except Exception as e:
            print(e)
            # if the button is not interactable, get the last page info
            soup = BeautifulSoup(driver.page_source,'html.parser')
            for i in soup.find_all('itx-product-view', {'ng-if':'::(product.state.isProduct || product.state.isFakeBanner)'}):
                if i.find('a', {'ng-bind':'::$ctrl.product.name'}).text != '':
                    product_name.append(i.find('a', {'ng-bind':'::$ctrl.product.name'}).text)  # product name
                    if i.find('span', {'class':'old-price text-14-m ng-binding ng-scope'}) == None:
                        # if it == None, No sale price, 
                        # if it != None, sale price is sale price, original price is original price is original price
                        original_price.append(i.find('span', {'class':'price text-14-m ng-binding ng-scope'}).text) # original
                        sale_price.append(None)
                    else:
                        original_price.append(i.find('span', {'class':'old-price text-14-m ng-binding ng-scope'}).text) # original
                        sale_price.append(i.find('span', {'class':'price text-14-m ng-binding ng-scope'}).text) # sale
                    product_url.append(i.find('a', {'ng-click':'$ctrl.goToProductDetail($event)'}).get('ng-href')) # product url
                    picture_url.append(i.find('img', {'class':'card9-image ng-scope'}).get('ng-src')) # pic url
                else:
                    product_name.append(None) # product name
                    original_price.append(None) # original price
                    sale_price.append(None) # sale price
                    product_url.append(None) # product url
                    picture_url.append(None) # picture url
            break
driver.close()

Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/itx-catalog/section/div/category-page/section[1]/div[2]/category-nav/div/div[2]/a"}
  (Session info: chrome=87.0.4280.88)



In [172]:
column_names = ['product_name', 'original_price', 'sale_price', 'product_link', 'image_link']
df = pd.DataFrame(columns = column_names)

df['product_name'] = product_name
df['original_price'] = original_price
df['sale_price'] = sale_price
df['product_link'] = product_url
df['image_link'] = picture_url
df

,product_name,original_price,sale_price,product_link,image_link
0,Polo sweater in 100% merino wool,$89.90,$69.90,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...
1,Crew neck sweater in 100% merino wool,$89.90,$69.90,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...
2,Crew neck sweater in 100% merino wool,$89.90,$69.90,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...
3,Wool/cashmere crew neck sweater,$89.90,$69.90,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...
4,Crew neck sweater in 100% merino wool,$89.90,$69.90,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...
...,...,...,...,...,...
853,100% cotton short sleeve t-shirt,$35.90,None,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...
854,100% cotton short sleeve t-shirt,$35.90,None,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...
855,100% cotton short sleeve t-shirt,$35.90,None,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...
856,100% cotton short sleeve t-shirt,$35.90,None,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...


In [173]:
df[(df['product_name']).apply(lambda x: str(x) == 'None')] # missing values

,product_name,original_price,sale_price,product_link,image_link
482,None,None,None,None,None
484,None,None,None,None,None
486,None,None,None,None,None
487,None,None,None,None,None
489,None,None,None,None,None
490,None,None,None,None,None
494,None,None,None,None,None
495,None,None,None,None,None
497,None,None,None,None,None
498,None,None,None,None,None


In [174]:
# drop rows which have na
droplist = df[(df['product_name']).apply(lambda x: str(x) == 'None')].index.tolist()
df = df.drop(droplist)
df

,product_name,original_price,sale_price,product_link,image_link
0,Polo sweater in 100% merino wool,$89.90,$69.90,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...
1,Crew neck sweater in 100% merino wool,$89.90,$69.90,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...
2,Crew neck sweater in 100% merino wool,$89.90,$69.90,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...
3,Wool/cashmere crew neck sweater,$89.90,$69.90,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...
4,Crew neck sweater in 100% merino wool,$89.90,$69.90,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...
...,...,...,...,...,...
853,100% cotton short sleeve t-shirt,$35.90,None,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...
854,100% cotton short sleeve t-shirt,$35.90,None,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...
855,100% cotton short sleeve t-shirt,$35.90,None,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...
856,100% cotton short sleeve t-shirt,$35.90,None,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...


# Second Layer

In [195]:
product_description = []
material = []
picture_name = []
color = []
brand = []
product_type = []
timestamp = []
counter = 0

driver = webdriver.Chrome(executable_path='/Users/fufufu/Documents/Python/Web Analytics/Selenium/chromedriver')
for link in df['product_link']:
    if(link) == product_url[0]: # only need to click for first url
        driver.get(link)
        time.sleep(3)
        driver.find_element_by_xpath("/html/body/div[2]/div[3]/div/div/div/div/div/div[2]/button").click() # close coookie noti
        soup = BeautifulSoup(driver.page_source,'html.parser')
        # product_description
        if soup.find_all('li', {'class':'description ng-scope'}) != []:
            a = ''
            for des in soup.find_all('li', {'class':'description ng-scope'}):
                a += des.text + '; '
            product_description.append(a[:-2])
        else:
            product_description.append(None)
        # material 
        if soup.find_all('div', {'ng-repeat':"compositionPart in $ctrl.compositionParts | orderBy: 'part'"}) != None:
            res = ''
            for mat1 in soup.find_all('div', {'ng-repeat':"compositionPart in $ctrl.compositionParts | orderBy: 'part'"}):
                res += ('; ' + mat1.find('h4', {'class':'h5 ng-binding'}).text + ':')
                for mat2 in mat1.find_all('span', {'class':'p1 ng-binding ng-scope'}):
                    res += mat2.text
            material.append(res[2:])
        else:
            material.append(None)
        # picture name
        picture_name.append('Massimo_Dutti_Men_' + str(counter) + '.jpg') 
        # color
        if soup.find('span', {'class':'epsilon ng-binding'}) != None:
            color.append(soup.find('span', {'class':'epsilon ng-binding'}).text)
        else:
            color.append(None)
        brand.append('Massimo Dutti') # brand
        product_type.append("Men's Clothing") # product type
        timestamp.append(date_today) # time_stamp 
    else:
        driver.get(link)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source,'html.parser')
        # product_description
        if soup.find_all('li', {'class':'description ng-scope'}) != []:
            a = ''
            for des in soup.find_all('li', {'class':'description ng-scope'}):
                a += des.text + '; '
            product_description.append(a[:-2])
        else:
            product_description.append(None)
        # material 
        if soup.find_all('div', {'ng-repeat':"compositionPart in $ctrl.compositionParts | orderBy: 'part'"}) != None:
            res = ''
            for mat1 in soup.find_all('div', {'ng-repeat':"compositionPart in $ctrl.compositionParts | orderBy: 'part'"}):
                res += ('; ' + mat1.find('h4', {'class':'h5 ng-binding'}).text + ':')
                for mat2 in mat1.find_all('span', {'class':'p1 ng-binding ng-scope'}):
                    res += mat2.text
            material.append(res[2:])
        else:
            material.append(None)
        # picture name
        picture_name.append('Massimo_Dutti_Men_' + str(counter) + '.jpg') 
        # color
        if soup.find('span', {'class':'epsilon ng-binding'}) != None:
            color.append(soup.find('span', {'class':'epsilon ng-binding'}).text)
        else:
            color.append(None)
        brand.append('Massimo Dutti') # brand
        product_type.append("Men's Clothing") # product type
        timestamp.append(date_today) # time_stamp 
    counter += 1 # update, for picture name

In [196]:
column_names = ["brand", "product_type", "product_name", 'product_description', "price", 
                "sales_price", "color","material", "product_url", "picture_url", "timestamp", 'picture_name']
second_pass = pd.DataFrame(columns = column_names)
second_pass.brand = brand
second_pass.product_type = product_type
second_pass.product_name = df['product_name']
second_pass.product_description = product_description
second_pass.price = df['original_price']
second_pass.sales_price = df['sale_price']
second_pass.color = color
second_pass.material = material
second_pass.product_url = df['product_link']
second_pass.picture_url = df['image_link']
second_pass.timestamp = timestamp
second_pass.picture_name = picture_name
second_pass

,brand,product_type,product_name,product_description,price,sales_price,color,material,product_url,picture_url,timestamp,picture_name
0,Massimo Dutti,Men's Clothing,Polo sweater in 100% merino wool,None,$89.90,$69.90,Navy blue,Outer:wool 100%,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...,2021-01-03,Massimo_Dutti_Men_0.jpg
1,Massimo Dutti,Men's Clothing,Crew neck sweater in 100% merino wool,Sweater; Merino wool is naturally anti-bacteri...,$89.90,$69.90,Black,Outer:wool 100%,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...,2021-01-03,Massimo_Dutti_Men_1.jpg
2,Massimo Dutti,Men's Clothing,Crew neck sweater in 100% merino wool,Sweater; Merino wool is naturally anti-bacteri...,$89.90,$69.90,Navy blue,Outer:wool 100%,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...,2021-01-03,Massimo_Dutti_Men_2.jpg
3,Massimo Dutti,Men's Clothing,Wool/cashmere crew neck sweater,Extra fine thread; Made from wool and cashmere...,$89.90,$69.90,Anthracite grey,"Outer:wool 90%, cashmere 10%",https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...,2021-01-03,Massimo_Dutti_Men_3.jpg
4,Massimo Dutti,Men's Clothing,Crew neck sweater in 100% merino wool,Sweater; Merino wool is naturally anti-bacteri...,$89.90,$69.90,Mole brown,Outer:wool 100%,https://www.massimodutti.com/us/men/sale/selec...,https://static.massimodutti.net/3/photos/2021/...,2021-01-03,Massimo_Dutti_Men_4.jpg
...,...,...,...,...,...,...,...,...,...,...,...,...
815,Massimo Dutti,Men's Clothing,Jogging fit cotton chino trousers,100% mercerised cotton; Casual; Crew neck; Sho...,$69.90,None,Green,Outer:cotton 100%,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...,2021-01-03,Massimo_Dutti_Men_815.jpg
816,Massimo Dutti,Men's Clothing,Jogging fit cotton chino trousers,Soft fabric that’s pleasant to the touch ; Mad...,$69.90,None,Cream,Outer:cotton 100%,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...,2021-01-03,Massimo_Dutti_Men_816.jpg
817,Massimo Dutti,Men's Clothing,Jogging fit cotton chino trousers,Soft fabric that’s pleasant to the touch ; Mad...,$69.90,None,Grey marl,Outer:cotton 100%,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...,2021-01-03,Massimo_Dutti_Men_817.jpg
818,Massimo Dutti,Men's Clothing,Extra slim fit chinos,Soft fabric that’s pleasant to the touch ; Mad...,$69.90,None,Dark salmon,Outer:cotton 100%,https://www.massimodutti.com/us/men/collection...,https://static.massimodutti.net/3/photos/2021/...,2021-01-03,Massimo_Dutti_Men_818.jpg


In [239]:
# drop rows which have nan, casue these rows are website promotions
droplist = second_pass[second_pass['picture_url'].apply(lambda x: str(x) == 'nan')].index.tolist()
second_pass = second_pass.drop(droplist)

In [240]:
second_pass.to_csv("Massimo_Dutti_Men.csv", index=False, encoding = 'utf-8-sig')

In [242]:
import os
import requests as req

new_pd = pd.read_csv('Massimo_Dutti_Men.csv', encoding='utf-8-sig')
4523
os.mkdir('Massimo_Dutti_Men')
i = 1
for index, link in enumerate(new_pd['picture_url']):
    img_data = req.get(link).content
    with open('Massimo_Dutti_Men/'+ "Massimo_Dutti_Men_"+str(index+1)+'.jpg','wb+') as f:
        f.write(img_data)
    i+=1